In [1]:
## Cálculo del tamaño de los bounding boxes

In [2]:
import torch
from ultralytics import YOLO
torch.has_mps
import torchvision

In [3]:
torch.__version__

'1.12.1'

In [4]:
torchvision.__version__

'0.13.1a0'

In [5]:
model_grapes = YOLO(r"C:\Users\danim\Documents\yolov8_grapes\runs\detect\grape_detector\weights\best.pt")
model_qr = YOLO(r"C:\Users\danim\Documents\yolov8_grapes\runs\detect\qr_detector\weights\best.pt")

In [9]:
images_path = r"C:\Users\danim\Documents\yolov8_grapes\dataset_qr\valid\images"

In [10]:
results_grapes = model_grapes.predict(source=images_path, save=False, save_txt=False, agnostic_nms=False, iou=0.25, line_width=5, conf=0.3, show_labels=True)


image 1/4 C:\Users\danim\Documents\yolov8_grapes\dataset_qr\valid\images\1597909019965_jpg.rf.94d33200a8e360888643563944931bf0.jpg: 800x608 4 grapes, 224.0ms
image 2/4 C:\Users\danim\Documents\yolov8_grapes\dataset_qr\valid\images\1597911008306_jpg.rf.4c0c6c9d29bef46c581f18c12e755d24.jpg: 800x608 9 grapes, 275.4ms
image 3/4 C:\Users\danim\Documents\yolov8_grapes\dataset_qr\valid\images\1597913693056_jpg.rf.927c3efe77b160783c349484c76be2b6.jpg: 800x608 7 grapes, 272.3ms
image 4/4 C:\Users\danim\Documents\yolov8_grapes\dataset_qr\valid\images\1597915138046_jpg.rf.605f53638a84d37e2986d1232064dc5a.jpg: 608x800 9 grapes, 248.9ms
Speed: 8.1ms preprocess, 255.1ms inference, 0.2ms postprocess per image at shape (1, 3, 800, 800)


In [11]:
results_qr = model_qr.predict(source=images_path, save=False, save_txt=False, agnostic_nms=False, iou=0.25, line_width=5, conf=0.01, show_labels=True)


image 1/4 C:\Users\danim\Documents\yolov8_grapes\dataset_qr\valid\images\1597909019965_jpg.rf.94d33200a8e360888643563944931bf0.jpg: 800x608 2 QRs, 288.2ms
image 2/4 C:\Users\danim\Documents\yolov8_grapes\dataset_qr\valid\images\1597911008306_jpg.rf.4c0c6c9d29bef46c581f18c12e755d24.jpg: 800x608 2 QRs, 210.6ms
image 3/4 C:\Users\danim\Documents\yolov8_grapes\dataset_qr\valid\images\1597913693056_jpg.rf.927c3efe77b160783c349484c76be2b6.jpg: 800x608 2 QRs, 265.5ms
image 4/4 C:\Users\danim\Documents\yolov8_grapes\dataset_qr\valid\images\1597915138046_jpg.rf.605f53638a84d37e2986d1232064dc5a.jpg: 608x800 2 QRs, 240.9ms
Speed: 7.8ms preprocess, 251.3ms inference, 2.7ms postprocess per image at shape (1, 3, 800, 800)


In [108]:
QR_AREA = 19 * 19 # cm^2

In [110]:
results_qr[0][1].boxes[0].xyxy.tolist()

import pandas as pd

df_qr = pd.DataFrame(columns=['image','area(pixels)', 'avg_area'])

image = results_qr[0][1].path.split('\\')[-1]

bbox = results_qr[0].boxes[0].xyxy.tolist()

#print(bbox)

len(results_qr[0].boxes)

for i in range(len(results_qr)):
    image = results_qr[i][1].path.split('\\')[-1]
    area = []
    for j in range(len(results_qr[i].boxes)):
        bbox = results_qr[i].boxes[j].xyxy.tolist()[0]
        area.append((bbox[2] - bbox[0]) * (bbox[3] - bbox[1]))
    avg_area = sum(area) / len(area)
    ratio = avg_area / QR_AREA
    df_qr = df_qr.append({'image': image, 'area(pixels)': area, 'avg_area': avg_area, 'ratio': ratio}, ignore_index=True)

df_qr.head(10)

C:\Users\danim\AppData\Local\Temp\ipykernel_19904\3647278603.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_qr = df_qr.append({'image': image, 'area(pixels)': area, 'avg_area': avg_area, 'ratio': ratio}, ignore_index=True)
C:\Users\danim\AppData\Local\Temp\ipykernel_19904\3647278603.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_qr = df_qr.append({'image': image, 'area(pixels)': area, 'avg_area': avg_area, 'ratio': ratio}, ignore_index=True)
C:\Users\danim\AppData\Local\Temp\ipykernel_19904\3647278603.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_qr = df_qr.append({'image': image, 'area(pixels)': area, 'avg_area': avg_area, 'ratio': ratio}, ignore_index=True)
C:\Users\danim\AppData\Local\Temp\ipy

,image,area(pixels),avg_area,ratio
0,1597909019965_jpg.rf.94d33200a8e36088864356394...,"[28366.198487393558, 27009.106378912926]",27687.652433,76.697098
1,1597911008306_jpg.rf.4c0c6c9d29bef46c581f18c12...,"[23996.003589630127, 19832.747802883387]",21914.375696,60.704642
2,1597913693056_jpg.rf.927c3efe77b160783c349484c...,"[18867.02387496829, 20868.272033929825]",19867.647954,55.035036
3,1597915138046_jpg.rf.605f53638a84d37e2986d1232...,"[21716.994204610586, 17623.704013824463]",19670.349109,54.488502


Ahora calculamos el area en píxeles para las uvas correspondientes a las fotos.

In [119]:
df_grapes = pd.DataFrame()

for i in range(len(results_grapes)):
    image = results_qr[i][1].path.split('\\')[-1]
    area = []
    for j in range(len(results_grapes[i].boxes)):
        bbox = results_grapes[i].boxes[j].xyxy.tolist()[0]
        area = (bbox[2] - bbox[0]) * (bbox[3] - bbox[1])
        ratio = df_qr[df_qr['image'] == image]['ratio'].values[0]
        area_cm = area / ratio
        df_grapes = df_grapes.append({'image': image, 'bbox': j, 'area(pixels)': area, 'ratio': ratio, 'area_cm^2': area_cm}, ignore_index=True)

df_grapes.head(10)

C:\Users\danim\AppData\Local\Temp\ipykernel_19904\3517526329.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_grapes = df_grapes.append({'image': image, 'bbox': j, 'area(pixels)': area, 'ratio': ratio, 'area_cm^2': area_cm}, ignore_index=True)
C:\Users\danim\AppData\Local\Temp\ipykernel_19904\3517526329.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_grapes = df_grapes.append({'image': image, 'bbox': j, 'area(pixels)': area, 'ratio': ratio, 'area_cm^2': area_cm}, ignore_index=True)
C:\Users\danim\AppData\Local\Temp\ipykernel_19904\3517526329.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_grapes = df_grapes.append({'image': image, 'bbox': j, 'area(pixels)': area, 'ratio': ratio, 'area_cm^2': area_cm},

,image,bbox,area(pixels),ratio,area_cm^2
0,1597909019965_jpg.rf.94d33200a8e36088864356394...,0,24508.350134,76.697098,319.547293
1,1597909019965_jpg.rf.94d33200a8e36088864356394...,1,15149.590997,76.697098,197.524957
2,1597909019965_jpg.rf.94d33200a8e36088864356394...,2,12621.103488,76.697098,164.557771
3,1597909019965_jpg.rf.94d33200a8e36088864356394...,3,14519.760504,76.697098,189.313036
4,1597911008306_jpg.rf.4c0c6c9d29bef46c581f18c12...,0,19465.704421,60.704642,320.662536
5,1597911008306_jpg.rf.4c0c6c9d29bef46c581f18c12...,1,12471.029190,60.704642,205.437819
6,1597911008306_jpg.rf.4c0c6c9d29bef46c581f18c12...,2,18242.800031,60.704642,300.517382
7,1597911008306_jpg.rf.4c0c6c9d29bef46c581f18c12...,3,29023.252343,60.704642,478.105981
8,1597911008306_jpg.rf.4c0c6c9d29bef46c581f18c12...,4,10074.247989,60.704642,165.955151
9,1597911008306_jpg.rf.4c0c6c9d29bef46c581f18c12...,5,17627.684639,60.704642,290.384460
